In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('\df_cleaned.csv')
submit = pd.read_csv('sample_submission.csv')
tfrec = pd.read_csv('tfsubmission.csv")

# Repurchased Items

In [3]:
re = df.groupby(['customer_id','article_id']).agg(count_item=('article_id','count'))
re = re.reset_index()
re = re[re['count_item']>10].copy()
re = re.sort_values(by='count_item', ascending=False).groupby('customer_id').head(1)
re['article_id'] = re['article_id'].astype(str).str.zfill(10)
re = re.drop('count_item', axis=1)
re.columns = ['customer_id', 're_item']

In [4]:
submit = pd.merge(submit, re, on='customer_id', how='left').fillna('')

# Last Purchased Items

In [5]:
last = df.sort_values(by='t_dat', ascending=False).groupby('customer_id').head(5)
last['article_id'] = last['article_id'].astype(str).str.zfill(10)
last = pd.DataFrame(last.groupby('customer_id')['article_id'].apply(list)).reset_index()
last['article_id'] = [' '.join(map(str, i)) for i in last['article_id']]
last.columns = ['customer_id','last_item']

In [6]:
submit = pd.merge(submit, last, on='customer_id', how='left').fillna('')

# Tensorflow recommenders

In [7]:
tfrec.columns = ['customer_id', 'tf']
submit = pd.merge(submit, tfrec, on='customer_id', how='left').fillna('')

# Last Week Popular Items

In [8]:
pop = df[df['t_dat']>'2020-09-15'].copy()
pop['article_id'] = pop['article_id'].astype(str).str.zfill(10)
itemlist = ' '.join(list(pop['article_id'].value_counts().head(12).index))

In [9]:
submit['pop'] = itemlist

# Combine recommendations

In [10]:
submit['merge'] = submit['re_item']+' '+submit['last_item']+' '+submit['tf']+' '+submit['pop']
submit = submit.drop(['re_item','last_item','prediction','tf','pop'], axis=1)
submit['merge'] = submit['merge'].apply(lambda x: x.split())
submit['merge'] = submit['merge'].apply(lambda x: list(sorted(set(x), key=x.index))).str[:12]
submit['merge'] = [' '.join(map(str, i)) for i in submit['merge']]
submit.columns = ['customer_id','prediction']